### **Crawl**

In [2]:
from urllib.request import urlopen, Request
import json

data = []

for i in range(1, 17):
    url = 'https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page=' + str(i) + '&id=3&order=view_count2&dir=desc&fearture=flashsale_samsung'

    try:
        rep = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        page = urlopen(rep, timeout=30).read()
        crawl_data = json.loads(page)
        data.extend(crawl_data)

    except:
        print(url + ': không tồn tại hoặc bị ẩn')
        #f_temp.write(url+'\n')



In [ ]:
data

In [6]:
data[1]

{'type_id': 'configurable',
 'entity_id': '16520',
 'name': 'iPhone 11 64GB I Chính hãng VN/A ',
 'url': 'https://cellphones.com.vn/iphone-11.html',
 'price': '18000000.0000',
 'final_price': 11490000,
 'small_image': 'https://cdn.cellphones.com.vn/media/catalog/product/cache/7/small_image/9df78eab33525d08d6e5fb8d27136e95/i/p/iphone_11_white_4_.png',
 'promotion_information': '<div class="promotion"><p class="gift-cont">[HOT] Thu cũ lên đời giá cao - Thủ tục nhanh - Trợ giá lên tới 1.000.000đ và <b>1 km</b> khác</p></div>',
 'count_all_reviews': 67,
 'average_ratings': 4.9,
 'attribute_black_friday': False,
 'attribute_uu_dai_tet': False,
 'attribute_doc_quyen': False,
 'tinh_trang_may_cu': None,
 'isLastPage': False,
 'stock_available': 46,
 'display_size': '6.1 inches',
 'laptop_ram': False,
 'hdd_sdd': False,
 'laptop_cpu': False,
 'memory_internal': '4 GB',
 'storage': '64 GB',
 'linh_kien_attribute': None}

In [ ]:
with open('Data\Cellphones\Cellphones_raw_data.json', "w", encoding="utf-8") as outfile:
        json.dump(data, outfile)

### **Tranform to csv**

In [1]:
import json

with open('Data\Cellphones\Cellphones_raw_data.json', "r", encoding="utf-8") as infile:
        data = json.load(infile)

len(data)

309

In [2]:
data[5]

{'type_id': 'configurable',
 'entity_id': '32364',
 'name': 'Samsung Galaxy S22 Ultra (8GB - 128GB)',
 'url': 'https://cellphones.com.vn/samsung-galaxy-s22-ultra.html',
 'price': '30990000.0000',
 'final_price': 29190000,
 'small_image': 'https://cdn.cellphones.com.vn/media/catalog/product/cache/7/small_image/9df78eab33525d08d6e5fb8d27136e95/s/m/sm-s908_galaxys22ultra_front_green_211119.jpg',
 'promotion_information': '<div class="promotion"><div class="promo-list"><ul><li class="pmh-title">PMH</li> <li class="pmh-content">2.000.000\xa0₫</li></ul></div><p class="coupon-price">Giảm thêm 4.000.000đ khi thu cũ đổi mới và <b>2 km</b> khác</p></div>',
 'count_all_reviews': 0,
 'average_ratings': 0,
 'attribute_black_friday': False,
 'attribute_uu_dai_tet': False,
 'attribute_doc_quyen': '1643',
 'tinh_trang_may_cu': None,
 'isLastPage': False,
 'stock_available': 46,
 'display_size': '6.8 inches',
 'laptop_ram': False,
 'hdd_sdd': False,
 'laptop_cpu': False,
 'memory_internal': '8 GB',
 's

In [3]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(data)
df

,type_id,entity_id,name,url,price,final_price,small_image,promotion_information,count_all_reviews,average_ratings,...,tinh_trang_may_cu,isLastPage,stock_available,display_size,laptop_ram,hdd_sdd,laptop_cpu,memory_internal,storage,linh_kien_attribute
0,configurable,28979,Samsung Galaxy A73 (5G),https://cellphones.com.vn/samsung-galaxy-a73.html,12990000.0000,10190000,https://cdn.cellphones.com.vn/media/catalog/pr...,"<div class=""promotion""><p class=""gift-cont"">Co...",0.0,0.0,...,None,False,46,6.7 inches,False,False,False,8 GB,256 GB,None
1,configurable,16520,iPhone 11 64GB I Chính hãng VN/A,https://cellphones.com.vn/iphone-11.html,18000000.0000,11490000,https://cdn.cellphones.com.vn/media/catalog/pr...,"<div class=""promotion""><p class=""gift-cont"">[H...",67.0,4.9,...,None,False,46,6.1 inches,False,False,False,4 GB,64 GB,None
2,configurable,33485,iPhone 13 Pro Max 128GB | Chính hãng VN/A,https://cellphones.com.vn/iphone-13-pro-max.html,34990000.0000,34990000,https://cdn.cellphones.com.vn/media/catalog/pr...,"<div class=""promotion""><p class=""gift-cont"">Th...",0.0,0.0,...,None,False,46,6.7 inches,False,False,False,6 GB,128 GB,None
3,configurable,33618,Xiaomi Redmi Note 11,https://cellphones.com.vn/xiaomi-redmi-note-11...,4690000.0000,4490000,https://cdn.cellphones.com.vn/media/catalog/pr...,"<div class=""promotion""><p class=""gift-cont"">Gi...",0.0,0.0,...,None,False,46,6.43 inches,False,False,False,4 GB,64 GB,None
4,configurable,25864,iPhone 12 Pro Max 128GB I Chính hãng VN/A,https://cellphones.com.vn/iphone-12-pro-max.html,32990000.0000,27000000,https://cdn.cellphones.com.vn/media/catalog/pr...,"<div class=""promotion""><p class=""gift-cont"">[H...",0.0,0.0,...,None,False,46,6.7 inches,False,False,False,6 GB,128 GB,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,configurable,24120,Xiaomi Mi Note 10 Lite 64GB,https://cellphones.com.vn/xiaomi-mi-note-10-li...,8990000.0000,0,https://cdn.cellphones.com.vn/media/catalog/pr...,"<div class=""promotion""></div>",NaN,NaN,...,None,False,56,6.47 inches,False,False,False,6 GB,64 GB,None
305,configurable,22461,Realme X50 Pro 5G,https://cellphones.com.vn/realme-x50-pro-5g.html,0.0000,0,https://cdn.cellphones.com.vn/media/catalog/pr...,None,NaN,NaN,...,Tin đồn,False,56,6.44 inches,False,False,False,6 GB,128 GB,None
306,configurable,17861,Oppo K3,https://cellphones.com.vn/oppo-k3.html,0.0000,0,https://cdn.cellphones.com.vn/media/catalog/pr...,None,NaN,NaN,...,Tin đồn,False,56,6.5 inches,False,False,False,6 GB,64 GB,None
307,configurable,14376,Xiaomi Mi Mix 4,https://cellphones.com.vn/xiaomi-mi-mix-4.html,0.0000,0,https://cdn.cellphones.com.vn/media/catalog/pr...,None,NaN,NaN,...,Tin đồn,False,56,False,False,False,False,False,False,None


In [4]:
cols_drop = ['isLastPage', 'laptop_ram', 'hdd_sdd', 'laptop_cpu', 'tinh_trang_may_cu', 'linh_kien_attribute']

df.drop(columns=cols_drop, inplace=True)

In [5]:
df['price'] = pd.to_numeric(df['price'], downcast='integer')
df['final_price'] = pd.to_numeric(df['final_price'], downcast='integer')
df['entity_id'] = pd.to_numeric(df['entity_id'])
df['name'] = df['name'].astype('string')

In [6]:
with pd.option_context('display.max_colwidth', None):
  display(df.query("count_all_reviews != count_all_reviews")[['name', 'price', 'final_price', 'stock_available','count_all_reviews', 'average_ratings', 'url']])

,name,price,final_price,stock_available,count_all_reviews,average_ratings,url
291,Samsung Galaxy Z Fold2 5G (Phiên bản mùa xuân),50000000,39000000,56,NaN,NaN,https://cellphones.com.vn/samsung-galaxy-z-fold-2-mua-xuan.html
292,iPhone SE 2020 256GB I Chính hãng VN/A,17990000,13010000,56,NaN,NaN,https://cellphones.com.vn/iphone-se-2020-256gb.html
293,Samsung Galaxy S9+ (Plus) 64GB,19990000,13090000,56,NaN,NaN,https://cellphones.com.vn/samsung-galaxy-s9plus-chinh-hang.html
294,Xiaomi Redmi Note 9 Pro 128GB,6990000,6200000,56,NaN,NaN,https://cellphones.com.vn/xiaomi-redmi-note-9-pro-128gb.html
295,Samsung Galaxy Z Fold2 5G (Phiên bản mùa hè),50000000,50000000,56,NaN,NaN,https://cellphones.com.vn/samsung-galaxy-z-fold-2-mua-he.html
296,Oppo F19 Pro Plus,0,0,56,NaN,NaN,https://cellphones.com.vn/oppo-f19-pro-plus.html
297,Điện thoại Energizer Energy E100,490000,0,56,NaN,NaN,https://cellphones.com.vn/energizer-energy-e100.html
298,Tecno Spark 7T 6GB 128GB,3490000,3490000,56,NaN,NaN,https://cellphones.com.vn/tecno-spark-7t-6gb-128gb.html
299,Samsung Galaxy A21s 6GB,4690000,4690000,56,NaN,NaN,https://cellphones.com.vn/samsung-galaxy-a21s-6gb.html
300,Xiaomi Redmi 8 3GB,2690000,2500000,56,NaN,NaN,https://cellphones.com.vn/xiaomi-redmi-8.html


In [7]:
with pd.option_context('display.max_colwidth', None):
  display(df.query("price == 0")[['name', 'price', 'final_price', 'stock_available','count_all_reviews', 'average_ratings', 'url']])

,name,price,final_price,stock_available,count_all_reviews,average_ratings,url
17,Xiaomi POCO X3 Pro,0,0,56,0.0,0.0,https://cellphones.com.vn/xiaomi-poco-x3-pro.html
28,Realme Narzo 50A,0,0,56,0.0,0.0,https://cellphones.com.vn/realme-narzo-50a.html
49,Realme GT Neo 3,0,0,56,0.0,0.0,https://cellphones.com.vn/realme-gt-neo-3.html
52,Redmi K40,0,0,56,0.0,0.0,https://cellphones.com.vn/xiaomi-redmi-k40.html
63,Realme Q3 Pro,0,0,56,0.0,0.0,https://cellphones.com.vn/realme-q3-pro.html
79,OPPO Find N,0,0,56,0.0,0.0,https://cellphones.com.vn/oppo-find-n.html
95,Redmi K50 Gaming Edition,0,0,56,0.0,0.0,https://cellphones.com.vn/redmi-k50-gaming-edition.html
105,Realme GT2 Pro,0,0,56,0.0,0.0,https://cellphones.com.vn/realme-gt-2-pro.html
109,Xiaomi Redmi K50,0,0,56,0.0,0.0,https://cellphones.com.vn/xiaomi-redmi-k50.html
178,Xiaomi Redmi K50 Pro,0,0,56,0.0,0.0,https://cellphones.com.vn/xiaomi-redmi-k50-pro.html


In [8]:
df.drop(df[df.price == 0].index, inplace=True)
df[['stock_available', 'count_all_reviews']] = df[['stock_available', 'count_all_reviews']].fillna(0)

In [9]:
with pd.option_context('display.max_colwidth', None):
  display(df.query("average_ratings != average_ratings")[['name', 'price', 'final_price', 'stock_available','count_all_reviews', 'average_ratings', 'url']])

,name,price,final_price,stock_available,count_all_reviews,average_ratings,url
291,Samsung Galaxy Z Fold2 5G (Phiên bản mùa xuân),50000000,39000000,56,0.0,NaN,https://cellphones.com.vn/samsung-galaxy-z-fold-2-mua-xuan.html
292,iPhone SE 2020 256GB I Chính hãng VN/A,17990000,13010000,56,0.0,NaN,https://cellphones.com.vn/iphone-se-2020-256gb.html
293,Samsung Galaxy S9+ (Plus) 64GB,19990000,13090000,56,0.0,NaN,https://cellphones.com.vn/samsung-galaxy-s9plus-chinh-hang.html
294,Xiaomi Redmi Note 9 Pro 128GB,6990000,6200000,56,0.0,NaN,https://cellphones.com.vn/xiaomi-redmi-note-9-pro-128gb.html
295,Samsung Galaxy Z Fold2 5G (Phiên bản mùa hè),50000000,50000000,56,0.0,NaN,https://cellphones.com.vn/samsung-galaxy-z-fold-2-mua-he.html
297,Điện thoại Energizer Energy E100,490000,0,56,0.0,NaN,https://cellphones.com.vn/energizer-energy-e100.html
298,Tecno Spark 7T 6GB 128GB,3490000,3490000,56,0.0,NaN,https://cellphones.com.vn/tecno-spark-7t-6gb-128gb.html
299,Samsung Galaxy A21s 6GB,4690000,4690000,56,0.0,NaN,https://cellphones.com.vn/samsung-galaxy-a21s-6gb.html
300,Xiaomi Redmi 8 3GB,2690000,2500000,56,0.0,NaN,https://cellphones.com.vn/xiaomi-redmi-8.html
301,Vsmart Star 4 2GB,2050000,0,56,0.0,NaN,https://cellphones.com.vn/vsmart-star-4-2gb.html


In [10]:
df['average_ratings'] = df['average_ratings'].fillna(0.0)
df['promotion_information'] = df['promotion_information'].fillna('')

In [11]:
len(df)

274

In [12]:
df.loc[3, ['promotion_information']]

promotion_information    <div class="promotion"><p class="gift-cont">Gi...
Name: 3, dtype: object

In [13]:
# delete html tags in promotion_information
# pip install beautifulsoup4
# pip install lxml

from bs4 import BeautifulSoup

IDs = list(df.index)

for i in range(len(IDs)):
    df.loc[IDs[i], ['promotion_information']] = BeautifulSoup(df['promotion_information'].iloc[i], "lxml").text

In [14]:
len(df['promotion_information'])

274

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274 entries, 0 to 308
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   type_id                 274 non-null    object 
 1   entity_id               274 non-null    int64  
 2   name                    274 non-null    string 
 3   url                     274 non-null    object 
 4   price                   274 non-null    int32  
 5   final_price             274 non-null    int32  
 6   small_image             274 non-null    object 
 7   promotion_information   274 non-null    object 
 8   count_all_reviews       274 non-null    float64
 9   average_ratings         274 non-null    float64
 10  attribute_black_friday  274 non-null    bool   
 11  attribute_uu_dai_tet    274 non-null    object 
 12  attribute_doc_quyen     274 non-null    object 
 13  stock_available         274 non-null    int64  
 14  display_size            274 non-null    ob

In [16]:
with pd.option_context('display.max_colwidth', None):
  display(df.promotion_information)

0                                                     Combo 02 Mã ưu đãi CGV trị giá 200.000đ
1      [HOT] Thu cũ lên đời giá cao - Thủ tục nhanh - Trợ giá lên tới 1.000.000đ và 1 km khác
2                                                            Thu cũ lên đời - Trợ giá 1 triệu
3                                                       Giảm 440.000đ khi thanh toán qua Moca
4      [HOT] Thu cũ lên đời giá cao - Thủ tục nhanh - Trợ giá lên tới 1.000.000đ và 1 km khác
                                                ...                                          
300                                                                                          
301                                                                                          
303                                                                                          
304                                                                                          
308                                                         

In [17]:
df.to_csv (r'Data\Cellphones\Cellphones_data.csv', index=False, header=True)